In [3]:
import numpy as np 
import pandas as pd 
import statistics as stats 
import math
import matplotlib.pyplot as plt

returns_data = pd.read_excel("gmo_analysis_data.xlsx", "returns (total)", usecols="B:C")
returns_data = returns_data.to_numpy()
returns_data = np.transpose(returns_data)

signal_data = pd.read_excel("gmo_analysis_data.xlsx", "signals", usecols="B:D")
signal_data = signal_data.to_numpy()
signal_data = np.transpose(signal_data)

dates = pd.read_excel("gmo_analysis_data.xlsx", "returns (total)", usecols="A", dtype="string")
dates = dates.to_numpy()
dates = np.transpose(dates)
dates = dates[0]

#point 172: 2011

def max_drawdown(column):
    start = -1 
    end = -1
    recovery = -1 
    maxDrawdown = 0
    cumprod = (1+column).cumprod()
    #print(cumprod)
    i = 0
    j = 0
    k = 0
    for i in range(142):
        for j in range(142 - i):
            if(-((cumprod[i] - cumprod[j + i]) / cumprod[i]) < maxDrawdown):
                maxDrawdown = -100*((cumprod[i] - cumprod[j + i]) / cumprod[i])
                start = i
                end = j + i
                for k in range(142 - j - i):
                    if(cumprod[i] < cumprod[i + j + k]):
                        recovery = i + j + k
                        break
    print("Max Drawdown:")
    print("Start: " + dates[start] + " " + str(cumprod[start]))
    print("End: " + dates[end] + " " + str(cumprod[end]))
    print("Recovery: " + dates[recovery] + " " + str(cumprod[recovery]))

def print_stats(column):
    mean = stats.mean(column)*12
    vol = stats.stdev(column)*math.sqrt(12)
    print("mean: " + str(round(mean,3)))
    print("vol: " + str(round(vol,3)))
    print("SR: " + str(round(mean/vol,3)))
    print("Min: " + str(round(np.min(column), 3)))
    print("VaR" + str(round(np.quantile(column, 0.05),3)))
    #max_drawdown(column)
    print(" ")
    
print("Inception to 2011:")
print_stats(returns_data[0][:172])
#linear_regression(returns_data[0][:172], returns_data[1][:172])

print("2012-Present")
print_stats(returns_data[0][172:]) 
#linear_regression(returns_data[0][172:], returns_data[1][172:])

print("All Time")
print_stats(returns_data[0])

Inception to 2011:
mean: 0.047
vol: 0.126
SR: 0.371
Min: -0.145
VaR-0.059
 
2012-Present
mean: 0.048
vol: 0.094
SR: 0.516
Min: -0.119
VaR-0.039
 
All Time
mean: 0.048
vol: 0.112
SR: 0.424
Min: -0.145
VaR-0.047
 


In [23]:
from sklearn.linear_model import LinearRegression

model_alphas = {}
model_betas = {}

def lagged_regression(spy, signal, words):
    spy = spy.reshape(-1,1)[1:320]
    if(signal.shape == (320, )):
        signal = signal.reshape(-1, 1)
    else: 
        signal = signal.transpose()
    signal = signal[0:319]
    model = LinearRegression().fit(signal, spy)
    Beta = model.coef_[0]
    Alpha = model.intercept_[0]
    model_alphas[words] = Alpha
    model_betas[words] = Beta
    
    print(words)
    print("Alpha: " + str(round(Alpha,4)))
    print("Beta(s): ")
    print(Beta)
    print("R^2: " + str(round(model.score(signal, spy),4)))
    print(" ")
    

lagged_regression(returns_data[1], signal_data[0], "DP")
lagged_regression(returns_data[1], signal_data[1], "EP")
lagged_regression(returns_data[1], signal_data, "All")

DP
Alpha: -0.013
Beta(s): 
[0.01160053]
R^2: 0.0092
 
EP
Alpha: -0.0034
Beta(s): 
[0.0025801]
R^2: 0.0051
 
All
Alpha: -0.0145
Beta(s): 
[ 0.00916129  0.00213655 -0.00100334]
R^2: 0.0141
 


In [87]:
def forecast(alpha, beta, signal, spy, words):
    counter = 0
    return_ = np.zeros((319))
    signal = signal.transpose()
    for point in signal[0:319]:
        forecast_spy = alpha[words]+np.sum(beta[words]*point)
        weight_ = 100*(forecast_spy)
        return_[counter] = (weight_*spy[counter + 1])
        counter += 1
    cum_return = np.cumprod(1+return_)
    print(words)
    print(cum_return[-1])
    model = LinearRegression().fit(return_.reshape(-1,1)[0:319], spy[1:320].reshape(-1,1))
    Beta = model.coef_[0]
    Alpha = model.intercept_[0]
    print("Mean Return: " + str(round(stats.mean(return_),4)))
    print(alpha)
    print("Alpha: " + str(round(Alpha,3)))
    print("Beta(s): ")
    print(Beta)
    print("R^2: " + str(round(model.score(return_.reshape(-1,1)[0:319], spy[1:320].reshape(-1,1)),4)))
    print(" ")
forecast(model_alphas, model_betas, signal_data[0], returns_data[1], "DP")
forecast(model_alphas, model_betas, signal_data[1], returns_data[1], "EP")
forecast(model_alphas, model_betas, signal_data, returns_data[1], "All")
print("SPY: ")
print(np.cumprod(1+returns_data[1])[-1])

DP
9.66643818854634
Mean Return: 0.0083
{'DP': -0.012976157379866845, 'EP': -0.00338447711413299, 'All': -0.014488842292508103}
Alpha: 0.002
Beta(s): 
[0.77925491]
R^2: 0.683
 
EP
8.735158612192913
Mean Return: 0.0075
{'DP': -0.012976157379866845, 'EP': -0.00338447711413299, 'All': -0.014488842292508103}
Alpha: -0.0
Beta(s): 
[1.1184601]
R^2: 0.8011
 
All
13.934511931675248
Mean Return: 0.0093
{'DP': -0.012976157379866845, 'EP': -0.00338447711413299, 'All': -0.014488842292508103}
Alpha: 0.001
Beta(s): 
[0.80763443]
R^2: 0.6364
 
SPY: 
9.934045212710515
